# Исследование надежности заемщиков.

**Какова цель исследования?** 

- Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок.

**На каких данных проводится исследование?**
- Входные данные от банка — статистика о платёжеспособности клиентов.

**Что нужно выяснить?**
- Есть ли зависимость между количеством детей и возвратом кредита в срок?
- Есть ли зависимость между семейным положением и возвратом кредита в срок?
- Есть ли зависимость между уровнем дохода и возвратом кредита в срок?
- Как разные цели кредита влияют на его возврат в срок?

**Что будем делать с результатами?**
- Результаты исследования будут учтены при построении модели кредитного скоринга — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

### Шаг 1. Обзор данных

In [ ]:
# импорт библиотеки pandas
import pandas as pd

In [ ]:
# чтение файла с данными и сохранение в df
df = pd.read_csv('/datasets/data.csv')

In [ ]:
# получение первых 10 строк таблицы df
df.head(5)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу


In [ ]:
# просмотр типа данных в датафрейме
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


В таблице 11 столбцов. Согласно документации к данным:

- `children` — количество детей в семье
- `days_employed` — общий трудовой стаж в днях
- `dob_years` — возраст клиента в годах
- `education` — уровень образования клиента
- `education_id` — идентификатор уровня образования
- `family_status` — семейное положение
- `family_status_id` — идентификатор семейного положения
- `gender` — пол клиента
- `income_type` — тип занятости
- `debt` — имел ли задолженность по возврату кредитов
- `total_income` — ежемесячный доход
- `purpose` — цель получения кредита


### Шаг 2.1 Заполнение пропусков

In [ ]:
# подсчёт пропусков
round((df.isna().sum()) * 100 / len(df), 2)

children             0.0
days_employed       10.1
dob_years            0.0
education            0.0
education_id         0.0
family_status        0.0
family_status_id     0.0
gender               0.0
income_type          0.0
debt                 0.0
total_income        10.1
purpose              0.0
dtype: float64

In [ ]:
df.isna().mean()

children            0.000000
days_employed       0.100999
dob_years           0.000000
education           0.000000
education_id        0.000000
family_status       0.000000
family_status_id    0.000000
gender              0.000000
income_type         0.000000
debt                0.000000
total_income        0.100999
purpose             0.000000
dtype: float64

Пропуски обнаружились в двух столбцах - общем трудовом стаже в днях (`days_employed`) и ежемесячном доходе (`total_income`). Пропуски по второму могут повлиять на анализ зависимости между уровнем дохода и возвратом кредита в срок. Их доля составляет 10%.

Пропущенные значения могли появиться по разным причинам. Например, из-за нежелания указывать данные в столбце `days_employed` именно в днях, а в столбце `total_income` - данные о доходах в принципе. Заполняющий мог просто упустить и не обратить внимание на колонки, возможно, их заполнение было необязательным условием. Также ошибки могли возникнуть при подготовке данных для аналитики и выгрузке из баз.

### Шаг 2.2 Проверка данных на аномалии и исправления.

В данных могут встречаться аномалии, которые стоит учесть при анализе. Попробуем оценить их наличие сначала для столбца `days_employed`.

In [ ]:
df.describe()

,children,days_employed,dob_years,education_id,family_status_id,debt,total_income
count,21525.000000,19351.000000,21525.000000,21525.000000,21525.000000,21525.000000,1.935100e+04
mean,0.538908,63046.497661,43.293380,0.817236,0.972544,0.080883,1.674223e+05
std,1.381587,140827.311974,12.574584,0.548138,1.420324,0.272661,1.029716e+05
min,-1.000000,-18388.949901,0.000000,0.000000,0.000000,0.000000,2.066726e+04
25%,0.000000,-2747.423625,33.000000,1.000000,0.000000,0.000000,1.030532e+05
50%,0.000000,-1203.369529,42.000000,1.000000,0.000000,0.000000,1.450179e+05
75%,1.000000,-291.095954,53.000000,1.000000,1.000000,0.000000,2.034351e+05
max,20.000000,401755.400475,75.000000,4.000000,4.000000,1.000000,2.265604e+06


Среди данных большое количество отрицательных значений в столбце `days_employed`. К тому же для дней - слишком большие цифры, возможно, данные представлены в другом измерении, например, в часах или минутах. В столбце `total_income` в целом значения огромные.

Сначала попробуем сделать из отрицательных чисел положительные.

In [ ]:
df.loc[df['days_employed'] < 0, 'days_employed'] = df['days_employed'].abs()

Как видим, отрицательных значений больше не осталось.

In [ ]:
df.describe()

,children,days_employed,dob_years,education_id,family_status_id,debt,total_income
count,21525.000000,19351.000000,21525.000000,21525.000000,21525.000000,21525.000000,1.935100e+04
mean,0.538908,66914.728907,43.293380,0.817236,0.972544,0.080883,1.674223e+05
std,1.381587,139030.880527,12.574584,0.548138,1.420324,0.272661,1.029716e+05
min,-1.000000,24.141633,0.000000,0.000000,0.000000,0.000000,2.066726e+04
25%,0.000000,927.009265,33.000000,1.000000,0.000000,0.000000,1.030532e+05
50%,0.000000,2194.220567,42.000000,1.000000,0.000000,0.000000,1.450179e+05
75%,1.000000,5537.882441,53.000000,1.000000,1.000000,0.000000,2.034351e+05
max,20.000000,401755.400475,75.000000,4.000000,4.000000,1.000000,2.265604e+06


С огромными числами в столбце `days_employed` нужно что-то сделать. Вероятно, это все же данные, указанные в часах. Посмотрим на примере нескольких чисел из столбца. Разделим на 24, чтобы узнать, сколько дней составил трудовой стаж и на 365, чтобы узнать показатель в годах.

In [ ]:
(19351.000000/24)/365

2.2090182648401826

In [ ]:
(401755.400475/24)/365

45.86248863869862

In [ ]:
(24.141633/24)/365

0.0027558941780821914

Получилось примерно 2 года, 45 лет и менее года трудового стажа. Теперь попробуем все числа перевести в дни, исходя из гипотезы, что сейчас в данных ошибка, и они указаны в часах. 

In [ ]:
df['days_employed'] = df['days_employed']/24

In [ ]:
df.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,351.569709,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,167.700156,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,234.309275,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,171.864467,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,14177.753002,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу


Итак, в столбце `days_employed` мы получили данные трудового стажа в днях. Учитывая то, что это количественные переменные, а данных, чтобы заполнить пропуски точными значениями, у нас нет, используем медиану. Она устойчива к выбросам, особенно, когда некоторые значения сильно выделяются среди большинства.

In [ ]:
#расчет медианы
age_avg = df['days_employed'].median()

#заполнение пропусков медианой
df['days_employed'] = df['days_employed'].fillna(age_avg)

In [ ]:
df.head(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,351.569709,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,167.700156,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,234.309275,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,171.864467,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,14177.753002,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,38.591076,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,119.966752,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,6.365815,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,288.744387,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,91.198185,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


In [ ]:
#проверка, исчезли ли пропуски
print(df['days_employed'].isna().sum())

0


### Шаг 2.3. Изменение типов данных.

Пропуски в столбце `total_income` также заполним медианой.

In [ ]:
#расчет медианы
total_income_avg = df['total_income'].median()

#заполнение пропусков медианой
df['total_income'] = df['total_income'].fillna(total_income_avg)

In [ ]:
df.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,351.569709,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,167.700156,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,234.309275,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,171.864467,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,14177.753002,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу


In [ ]:
#перевод из вещественного числа в целочисленный
df['total_income'] = df['total_income'].astype('int')
df.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,351.569709,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья
1,1,167.700156,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля
2,0,234.309275,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья
3,3,171.864467,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование
4,0,14177.753002,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу


Проверим, произошла ли замена, выбрав случайную строку с данными

In [ ]:
total_income_check = df['total_income']

print(type(total_income_check[567]))

<class 'numpy.int64'>


### Шаг 2.4. Удаление дубликатов.

#### Проверка дубликатов в столбцах

Чтобы посмотреть количество повторов, воспользуемся методом `duplicated().sum()`. Его применение даст общее представление о количестве дубликатов. Для того, чтобы узнать, какие именно повторы и в каком регистре встречаются в датафрейме, используем метод `value_counts()`. Так, мы узнаем, какие повторы и в каком количестве встретились в столбце.

##### В столбце 'education'

In [ ]:
print ('Дубликатов в столбце:', df['education'].duplicated().sum())
print()
print(df['education'].value_counts())

Дубликатов в столбце: 21510

среднее                13750
высшее                  4718
СРЕДНЕЕ                  772
Среднее                  711
неоконченное высшее      668
ВЫСШЕЕ                   274
Высшее                   268
начальное                250
Неоконченное высшее       47
НЕОКОНЧЕННОЕ ВЫСШЕЕ       29
НАЧАЛЬНОЕ                 17
Начальное                 15
ученая степень             4
УЧЕНАЯ СТЕПЕНЬ             1
Ученая степень             1
Name: education, dtype: int64


In [ ]:
#приведем к нижнему регистру все дубликаты столбца 'education'
df['education'] = df['education'].str.lower()

In [ ]:
df['education'].value_counts()

среднее                15233
высшее                  5260
неоконченное высшее      744
начальное                282
ученая степень             6
Name: education, dtype: int64

##### В столбце 'family_status'

In [ ]:
print(df['family_status'].value_counts())

женат / замужем          12380
гражданский брак          4177
Не женат / не замужем     2813
в разводе                 1195
вдовец / вдова             960
Name: family_status, dtype: int64


##### В столбце 'income_type'

In [ ]:
df['income_type'].value_counts()

сотрудник          11119
компаньон           5085
пенсионер           3856
госслужащий         1459
безработный            2
предприниматель        2
в декрете              1
студент                1
Name: income_type, dtype: int64

##### В столбце 'purpose'

In [ ]:
df['purpose'].value_counts().sort_values(ascending = False)

свадьба                                   797
на проведение свадьбы                     777
сыграть свадьбу                           774
операции с недвижимостью                  676
покупка коммерческой недвижимости         664
покупка жилья для сдачи                   653
операции с жильем                         653
операции с коммерческой недвижимостью     651
жилье                                     647
покупка жилья                             647
покупка жилья для семьи                   641
строительство собственной недвижимости    635
недвижимость                              634
операции со своей недвижимостью           630
строительство жилой недвижимости          626
покупка недвижимости                      624
покупка своего жилья                      620
строительство недвижимости                620
ремонт жилью                              612
покупка жилой недвижимости                607
на покупку своего автомобиля              505
заняться высшим образованием      

##### В столбце 'children'

In [ ]:
df['children'].value_counts()

 0     14149
 1      4818
 2      2055
 3       330
 20       76
-1        47
 4        41
 5         9
Name: children, dtype: int64

Значение -1 заменим на положительное, это явная ошибка.

In [ ]:
df['children'] = df['children'].replace(-1, 1)

В теории у женщины может родиться 20 детей, однако считаю, что это все же ошибка при выгрузке, поэтому можно удалить. Значений слишком мало, глобально на выборку они не влияют.

In [ ]:
df= df.drop(df[df['children'] ==20].index)

In [ ]:
df['children'].value_counts()

0    14149
1     4865
2     2055
3      330
4       41
5        9
Name: children, dtype: int64

##### В столбце 'debt'

In [ ]:
df['debt'].value_counts()

0    19716
1     1733
Name: debt, dtype: int64

##### В столбце 'gender'

In [ ]:
df['gender'].value_counts()

F      14189
M       7259
XNA        1
Name: gender, dtype: int64

In [ ]:
#посмотрим на строчку со значением XNA в столбце `gender`
df.loc[df.loc[:, 'gender'] == 'XNA']

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
10701,0,98.275021,24,неоконченное высшее,2,гражданский брак,1,XNA,компаньон,0,203905,покупка недвижимости


Значение XNА мы определить не можем, поэтому придется его удалить и не учитывать при конечных результатах.

In [ ]:
df= df.drop(df[df['gender'] == 'XNA'].index)

In [ ]:
df['gender'].value_counts()

F    14189
M     7259
Name: gender, dtype: int64

#### Проверка устранения дубликатов

In [ ]:
print ('Дубликатов в таблице:', df.duplicated().sum())

Дубликатов в таблице: 71


In [ ]:
#удалим оставшиеся дубликаты
df = df.drop_duplicates().reset_index(drop=True)

### Создание новых датафреймов

Создадим два новых датафрейма, в которых:
- каждому уникальному значению из education соответствует уникальное значение education_id — в первом;
- каждому уникальному значению из family_status соответствует уникальное значение family_status_id — во втором.


In [ ]:
education_df = df[['education', 'education_id']]

In [ ]:
education_df

,education,education_id
0,высшее,0
1,среднее,1
2,среднее,1
3,среднее,1
4,среднее,1
...,...,...
21372,среднее,1
21373,среднее,1
21374,среднее,1
21375,среднее,1


In [ ]:
education_df.duplicated().sum()

21372

In [ ]:
education_df.value_counts()

education            education_id
среднее              1               15112
высшее               0                5236
неоконченное высшее  2                 741
начальное            3                 282
ученая степень       4                   6
dtype: int64

In [ ]:
education_df = education_df.drop_duplicates().reset_index(drop=True)

In [ ]:
print(education_df)

             education  education_id
0               высшее             0
1              среднее             1
2  неоконченное высшее             2
3            начальное             3
4       ученая степень             4


In [ ]:
family_df = df[['family_status', 'family_status_id']]

In [ ]:
family_df

,family_status,family_status_id
0,женат / замужем,0
1,женат / замужем,0
2,женат / замужем,0
3,женат / замужем,0
4,гражданский брак,1
...,...,...
21372,гражданский брак,1
21373,женат / замужем,0
21374,гражданский брак,1
21375,женат / замужем,0


In [ ]:
family_df.duplicated().sum()

21372

In [ ]:
family_df.value_counts()

family_status          family_status_id
женат / замужем        0                   12290
гражданский брак       1                    4138
Не женат / не замужем  4                    2801
в разводе              3                    1193
вдовец / вдова         2                     955
dtype: int64

In [ ]:
family_df = family_df.drop_duplicates().reset_index(drop=True)

In [ ]:
print(family_df)

           family_status  family_status_id
0        женат / замужем                 0
1       гражданский брак                 1
2         вдовец / вдова                 2
3              в разводе                 3
4  Не женат / не замужем                 4


Удалим из исходного датафрейма столбцы education и family_status, оставив только их идентификаторы: education_id и family_status_id.

In [ ]:
df = df.drop(columns=['education', 'family_status'])

In [ ]:
df.head()

,children,days_employed,dob_years,education_id,family_status_id,gender,income_type,debt,total_income,purpose
0,1,351.569709,42,0,0,F,сотрудник,0,253875,покупка жилья
1,1,167.700156,36,1,0,F,сотрудник,0,112080,приобретение автомобиля
2,0,234.309275,33,1,0,M,сотрудник,0,145885,покупка жилья
3,3,171.864467,32,1,0,M,сотрудник,0,267628,дополнительное образование
4,0,14177.753002,53,1,1,F,пенсионер,0,158616,сыграть свадьбу


### Шаг 2.6. Категоризация дохода.

На основании диапазонов, указанных ниже, создадим столбец total_income_category с категориями:
- 0–30000 — 'E';
- 30001–50000 — 'D';
- 50001–200000 — 'C';
- 200001–1000000 — 'B';
- 1000001 и выше — 'A'.

Например, кредитополучателю с доходом 25000 нужно назначить категорию 'E', а клиенту, получающему 235000, — 'B'.

In [ ]:
def total_income_group(row):
    
    income = row['total_income']
    
    if 0 <= income <= 30000:
            return 'E'
    
    if 30001 <= income <= 50000:
            return 'D'
    
    if 50001 <= income <= 200000:
            return 'C'
    
    if 200001 <= income <= 1000000:
            return 'B'
        
    if income >= 1000001:
            return 'A'
        
    return 'доход неизвестен'

row_value = 348625684684 #проверочное значение дохода 
row_column = ['total_income'] #название столбца
row = pd.Series(data=row_value, index=row_column) #формирование строки 

print(total_income_group(row))


A


In [ ]:
df['total_income_category'] = df.apply(total_income_group, axis=1) #добавляем столбец в датафрейм
df.head()

,children,days_employed,dob_years,education_id,family_status_id,gender,income_type,debt,total_income,purpose,total_income_category
0,1,351.569709,42,0,0,F,сотрудник,0,253875,покупка жилья,B
1,1,167.700156,36,1,0,F,сотрудник,0,112080,приобретение автомобиля,C
2,0,234.309275,33,1,0,M,сотрудник,0,145885,покупка жилья,C
3,3,171.864467,32,1,0,M,сотрудник,0,267628,дополнительное образование,B
4,0,14177.753002,53,1,1,F,пенсионер,0,158616,сыграть свадьбу,C


In [ ]:
print(df['total_income_category'].value_counts()) #смотрим распределение категорий дохода

C    15960
B     5020
D      350
A       25
E       22
Name: total_income_category, dtype: int64


Судя по цифрам, большинство людей зарабатывает в диапазоне от 50 до 1 млн рублей. 

### Шаг 2.7. Категоризация целей кредита.

Создадим функцию, которая на основании данных из столбца purpose сформирует новый столбец purpose_category, в который войдут следующие категории:
- 'операции с автомобилем',
- 'операции с недвижимостью',
- 'проведение свадьбы',
- 'получение образования'.

Например, если в столбце purpose находится подстрока 'на покупку автомобиля', то в столбце purpose_category должна появиться строка 'операции с автомобилем'.

Для этой цели посмотрим уникальные значения в столбце `purpose`, которые помогут сформировать представления о общих целях кредитования. Значения по смыслу явно пересекаются и объединены общими тематиками, которые неплохо привести к единству. Для этого посмотрим, какие уникальные тематики представлены в таблице, а затем на их основе переименуем каждую.

In [ ]:
#df['purpose'].unique()

Как видим, среди общих тематик встречаются подготовка к свадьбе, покупки авто и жилья и т.д. Приведем цели с одинаковым смыслом к короткому названию. Для этого создадим функцию, которая распределит их по категориям:
- проведение свадьбы
- получение образования
- операции с автомобилем
- операции с недвижимостью 

In [ ]:
def categorize_purpose(row):
    try:
        if 'автом' in row:
            return 'операции с автомобилем'
        elif 'жил' in row or 'недвиж' in row:
            return 'операции с недвижимостью'
        elif 'свад' in row:
            return 'проведение свадьбы'
        elif 'образов' in row:
            return 'получение образования'
    except:
        return 'нет категории'

df['purpose_category'] = df['purpose'].apply(categorize_purpose) #добавляем столбец в датафрейм

In [ ]:
df.head()

,children,days_employed,dob_years,education_id,family_status_id,gender,income_type,debt,total_income,purpose,total_income_category,purpose_category
0,1,351.569709,42,0,0,F,сотрудник,0,253875,покупка жилья,B,операции с недвижимостью
1,1,167.700156,36,1,0,F,сотрудник,0,112080,приобретение автомобиля,C,операции с автомобилем
2,0,234.309275,33,1,0,M,сотрудник,0,145885,покупка жилья,C,операции с недвижимостью
3,3,171.864467,32,1,0,M,сотрудник,0,267628,дополнительное образование,B,получение образования
4,0,14177.753002,53,1,1,F,пенсионер,0,158616,сыграть свадьбу,C,проведение свадьбы


In [ ]:
print(df['purpose_category'].value_counts()) #смотрим распределение категорий цели

операции с недвижимостью    10774
операции с автомобилем       4290
получение образования        3998
проведение свадьбы           2315
Name: purpose_category, dtype: int64


Клиенты чаще заинтересованы в недвижимости, а на проведение свадьбы кредиты получает меньшинство

### Ответы на вопросы.

#### Вопрос 1: Есть ли зависимость между количеством детей и возвратом кредита в срок?

In [ ]:
#создадим новый датафрейм для выявления коэффициента зависимости
children_debt = pd.DataFrame()

#посчитаем и суммируем количество детей, сгруппировав их по колонкам датафрейма
children_debt['children_count'] = df.groupby('children')['debt'].count()
children_debt['children_sum'] = df.groupby('children')['debt'].sum()

#рассчитаем коэффициент зависимости
children_debt['children_result'] = children_debt['children_sum'] / children_debt['children_count']
children_debt.sort_values('children_result', ascending = False)

,children_count,children_sum,children_result
children,,,
4,41,4,0.097561
2,2052,194,0.094542
1,4855,445,0.091658
3,330,27,0.081818
0,14090,1063,0.075444
5,9,0,0.000000


Люди, не имеющие детей, реже имеют долги по кредитам. Думаю, зависимость здесь есть. Платить труднее тем, у кого есть дети.

#### Вопрос 2: Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [ ]:
#создадим новый датафрейм для выявления коэффициента зависимости
family_debt = pd.DataFrame()

#посчитаем и суммируем количество семенйных статусов, сгруппировав их по колонкам датафрейма
family_debt['family_count'] = df.groupby('family_status_id')['debt'].count()
family_debt['family_sum'] = df.groupby('family_status_id')['debt'].sum()

#рассчитаем коэффициент зависимости
family_debt['family_result'] = family_debt['family_sum'] / family_debt['family_count']
family_debt.sort_values('family_result', ascending = False)

,family_count,family_sum,family_result
family_status_id,,,
4,2801,273,0.097465
1,4138,385,0.093040
0,12290,928,0.075509
3,1193,84,0.070411
2,955,63,0.065969


In [ ]:
print(family_df.value_counts())

family_status          family_status_id
Не женат / не замужем  4                   1
в разводе              3                   1
вдовец / вдова         2                   1
гражданский брак       1                   1
женат / замужем        0                   1
dtype: int64


Чаще всего не возвращают кредиты в срок люди, которые не обременены условиями брака. Хотя, на мой взгляд, зависимость здесь небольшая. Грешат задолженностями и те, и те другие в схожих масштабах.

#### Вопрос 3: Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [ ]:
#создадим новый датафрейм для выявления коэффициента зависимости
income_debt = pd.DataFrame()

#посчитаем и суммируем количество доходов, сгруппировав их по колонкам датафрейма
income_debt['income_count'] = df.groupby('total_income_category')['debt'].count()
income_debt['income_sum'] = df.groupby('total_income_category')['debt'].sum()

#рассчитаем коэффициент зависимости
income_debt['income_result'] = income_debt['income_sum'] / income_debt['income_count']
income_debt.sort_values('income_result', ascending = False)

,income_count,income_sum,income_result
total_income_category,,,
E,22,2,0.090909
C,15960,1354,0.084837
A,25,2,0.080000
B,5020,354,0.070518
D,350,21,0.060000


- 0–30000 — 'E';
- 30001–50000 — 'D';
- 50001–200000 — 'C';
- 200001–1000000 — 'B';
- 1000001 и выше — 'A'.

Судя по результатам, как таковой зависимости нет. Поскольку странно, что люди, имеющие доход в пределах категории D реже имеют задолженность, чем те, кто получает доходы выше - категории C или А. Возможно, это связано с тем, что у людей с более высоким заработком и траты выше, следовательно, им не менее трудно выплачивать кредит.

#### Вопрос 4: Как разные цели кредита влияют на его возврат в срок?

In [ ]:
#создадим новый датафрейм для выявления коэффициента зависимости
purpose_debt = pd.DataFrame()

#посчитаем и суммируем количество целей, сгруппировав их по колонкам датафрейма
purpose_debt['purpose_count'] = df.groupby('purpose_category')['debt'].count()
purpose_debt['purpose_sum'] = df.groupby('purpose_category')['debt'].sum()

#рассчитаем коэффициент зависимости
purpose_debt['purpose_result'] = purpose_debt['purpose_sum'] / purpose_debt['purpose_count']
purpose_debt.sort_values('purpose_result', ascending = False)

,purpose_count,purpose_sum,purpose_result
purpose_category,,,
операции с автомобилем,4290,401,0.093473
получение образования,3998,369,0.092296
проведение свадьбы,2315,183,0.079050
операции с недвижимостью,10774,780,0.072397


Кредиты, которые берут на разовые события вроде свадьбы или покупки недвижимости, возвращают чаще, чем на длительные процессы вроде получения образования и операций с авто. 

## Общий вывод:

**Шаги, которые были выполнены по проекту:**
- На первом этапе был произведен обзор данных: подсчитаны пропуски, аномалии, дубликаты. Пропущенные значения количественных переменных были заменены медианой, устойчивой к выбросам. Аномалии обработаны, а дубликаты удалены.
- На втором этапе удалось категоризировать данные по уровню дохода и целям кредитования. 
- На третьем этапе были рассчитаны коэффициенты зависимости по каждому вопросу заказчика.

**Выводы по результатам исследования:**
- На степень задолженности по кредитам влияет количество людей в семье - то есть наличие партнеров и детей. Те, кто их не имеет платят намного быстрее, что, наверное, связано с тем, что доход легче распределять только на собственные нужды, а не на дополнительное число членов семьи. 
- Люди, берущие кредит на разовую покупку, возвращают его гораздо быстрее, чем те, кто склонен выплачивать частями в результате длительного процесса.
- Вне зависимости от уровня дохода люди могут быть не дисциплинированы в выплатах кредита в срок. В целом считаю, что доход не влияет на формирование долга.